## A brief history of data-parallel compute engines

This short lecture introduces map/reduce which has led to an ecosystem of data-parallel processing engines.  Notable, dask is the only engine in this list that doesn't build on top of the map/reduce paradigm.

### Map/Reduce (2004): 

At Google, Jeff Dean and Sanjay Ghemawat outline the future of large-scale data processing
  * <code>map()</code>: applies a user-defined function to a data partition and outputs a <code>key</code> used to identify objects in a class and a <code>value</code> that contains the data.
  * <code>reduce()</code>: takes all items with the same key and applies a user-defined function that aggregates the data.
  * This was the start of automatic parallelism (some disagree). The programmer writes two _pure_ functions and creates a computation that scales to thousands of nodes and GB of data.

<img src="https://cdn-images-1.medium.com/max/1600/1*KKm4roOpsum147kKk5qp7A.jpeg" width=512 />

This introduce the concept of key/value processing.  Here's some pseduocode and a visualization of how it works.

### Map/Reduce Example

Wordcount example from the original Google paper. Produce a count of the occurrence of each word in a set of documents.

```python
    map ( String key, String value ):
        // key: document name (file name)
        // value: document contents
        for each word w in value:
            EmitIntermediate ( word, "1" );
```

Mapper outputs `key=word, value="1"` for each word. Note that the output key and input key are different.

```python
    reduce ( String key, Iterator values ):
        // key: a word
        // value: a list of counts
        int result = 0;
        for each word v in values:
            result += ParsseInt ( v );
        EmitAsString ( result );
```
Reducer sums the counts of words.  Some properties:
  * reducer gets a list of values at a key
  * reduce cannot change the key, emits a value, that is reduced from list
  * user defined function
  
### WordCount Visualized

<img src="https://www.researchgate.net/profile/Oscar_Pereira3/publication/270448794/figure/fig6/AS:295098651824130@1447368409317/Word-count-program-flow-executed-with-MapReduce-5.png" width=768 title="from Oscar Perreira @ ResearchGate" />

Map/reduce is a data-parallel, __streaming__ processing engine.  
 * input files are read sequentially from disk. 
 * output files are written sequentially to disk. 
Good for many analysis tasks, but not good for iterative computations that reuse the output of one computation as input to the next.

Still the preferred programming engine for:
      * Web-log processing
      * Reverse web-link graph
      * Term vectors per host
      * Inverted index
      * Distributed sort
      
Distributed sort is what happens with a NULL mapper and a single NULL reducer.

### Hadoop! (2006):

Open source implementation of map/reduce computing
  * Credit to Doug Cutting and Mike Cafarella.
  * Users write Java functions that execute at scale.
  
We now move out of the Google ecosystem. Google has continued to make important contributions that inform open-source.

### Pig (2008): Meta-programming for Hadoop!

  * declarative constructs that compile to map/reduce programs
  * adopts the bag data type as an abstraction for key/value data 
  
```pig
lines = LOAD '/user/hadoop/HDFS_File.txt' AS (line:chararray);
words = FOREACH lines GENERATE FLATTEN(TOKENIZE(line)) as word;
grouped = GROUP words BY word;
wordcount = FOREACH grouped GENERATE group, COUNT(words);
DUMP wordcount;
```
  
### Hive (2010): An SQL(-like) interface to Hadoop!

  * Most SQL queries can be executed in two iterations of map/reduce
  * Similar to dask/pandas, Hive only implemented the part of SQL that translates to M/R

```sql
CREATE TABLE word_counts AS
SELECT word, count(1) AS count FROM
(SELECT explode(split(line, ' ')) AS word FROM FILES) w
GROUP BY word
ORDER BY word;
```

### Spark (2014): 
In-memory data for iterative programming in map/reduce

### Dask (2016?): 
"pythonic" version of Spark that eases programming for NumPy and pandas.